In [119]:
import os
import re
from datetime import datetime
import pandas as pd

In [120]:
with open(os.path.join("downloads/2024-11-08","ar_České Budějovice_Praha_D6_2024-11-08_07-42-14.html"), "r", encoding="utf-8") as spojeni:
    spojeni = spojeni.read()

In [121]:
with open(os.path.join("pokusy","ar.html"), "w+", encoding="utf-8") as l:
    l.write("\n".join(spojeni.split(">")))

In [122]:
def oscrapuj_ar(slozka, soubor):
    with open(os.path.join(slozka,soubor), "r", encoding="utf-8") as spojeni:
        spojeni = spojeni.read()
        spoje = []
        spoj = None
        oscrapovano = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', soubor).group(0)
        oscrapovano = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
        for index, radek in enumerate(spojeni.split(">")):
            if re.search(r"\d{1,2}\.\s\w{4,15}\s20\d{2}", radek):
                datum = re.search("\d{1,2}\.\s\w{4,15}\s20\d{2}", radek).group(0)
                datum = datum.replace("leden","1.").replace("únor","2.").replace("březen","3.").replace("duben","4.").replace("květen","5.").replace("červen","6.").replace('červenec','7.').replace('srpen','8.').replace('září','9.').replace('říjen','10.').replace('listopad','11.').replace('prosinec','12.').replace(' ','')
            if 'class="departure"' in radek:
                if spoj:
                    spoje.append(spoj)
                spoj = {}
                spoj['vlaky'] = []
                spoj['prodejce'] = 'ARR'
                spoj['prostredek'] = 'vlak'
                spoj['odkud'] = spojeni.split(">")[index + 1].split("<")[0]
                spoj['oscrapovano'] = oscrapovano
            if '<div class="arrival"' in radek:
                spoj['kam'] = spojeni.split(">")[index + 1].split("<")[0]
            if '<div class="time"' in radek:
                cas = spojeni.split(">")[index + 2].split("<")[0]
                spoj['odjezd'] = datetime.strptime('.'.join(datum.split('.')) + " " + cas.strip(), "%d.%m.%Y %H:%M")
                spoj['predstih'] = spoj['odjezd'] - oscrapovano
            if 'Vzdálenost' in radek:
                spoj['vzdalenost'] = int(re.search(r"\d{1,4}",radek).group())
            if ("hodin" in radek) and ("minut" in radek):
                jizdni_doba = radek.split(' a ')
                hodiny = re.search(r"\d{1,2}", jizdni_doba[0]).group()
                minuty = re.search(r"\d{1,2}", jizdni_doba[1]).group()
                spoj['jizdni_doba'] = (int(hodiny) * 60) + int(minuty)
            if "&nbsp;Kč" in radek:
                spoj['cena'] = int(re.search(r"\d{1,5}", radek.split("&nbsp;Kč")[0]).group())
            if 'class="line-number" ' in radek:
                spoj['vlaky'].append(spojeni.split(">")[index + 1].split("<")[0])
            if '<span class="priceFreeSeats"' in radek:
                spoj['volnych_mist'] = int(re.search(r'\d{1,4}', spojeni.split(">")[index + 1]).group())
    return spoje

<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\micha\AppData\Local\Temp\ipykernel_32304\262362362.py:10: SyntaxWarning: invalid escape sequence '\d'
  datum = re.search("\d{1,2}\.\s\w{4,15}\s20\d{2}", radek).group(0)


In [123]:
oscrapuj_ar("downloads/2024-11-08","ar_České Budějovice_Praha_D6_2024-11-08_07-42-14.html")

[{'vlaky': ['R 1167'],
  'prodejce': 'ARR',
  'prostredek': 'vlak',
  'odkud': 'České Budějovice',
  'oscrapovano': datetime.datetime(2024, 11, 8, 7, 42, 14),
  'kam': 'Praha hl.n.',
  'odjezd': datetime.datetime(2024, 11, 8, 12, 33),
  'predstih': datetime.timedelta(seconds=17446),
  'vzdalenost': 187,
  'jizdni_doba': 201,
  'cena': 200,
  'volnych_mist': 35},
 {'vlaky': ['R 1169'],
  'prodejce': 'ARR',
  'prostredek': 'vlak',
  'odkud': 'České Budějovice',
  'oscrapovano': datetime.datetime(2024, 11, 8, 7, 42, 14),
  'kam': 'Praha hl.n.',
  'odjezd': datetime.datetime(2024, 11, 8, 16, 43),
  'predstih': datetime.timedelta(seconds=32446),
  'vzdalenost': 187,
  'jizdni_doba': 211,
  'cena': 200,
  'volnych_mist': 18},
 {'vlaky': ['R 1171'],
  'prodejce': 'ARR',
  'prostredek': 'vlak',
  'odkud': 'České Budějovice',
  'oscrapovano': datetime.datetime(2024, 11, 8, 7, 42, 14),
  'kam': 'Praha hl.n.',
  'odjezd': datetime.datetime(2024, 11, 8, 18, 43),
  'predstih': datetime.timedelta(se

In [131]:
kam = "data" 
os.makedirs(kam, exist_ok=True)
hotove = [y for y in os.listdir(kam) if y[0:3] == "ar_"]
for x in os.listdir("downloads"):
    nazev_souboru = "ar_" + x + ".parquet"
    if nazev_souboru not in hotove:
        den = []
        ar = [y for y in os.listdir(f"downloads/{x}") if y[0:3] == "ar_"] 
        print(f"{x}: {len(ar)}")
        for y in ar:
            print(y)
            den = den + oscrapuj_ar(f"downloads/{x}",y)
        if len(den) > 0:
            df_den = pd.DataFrame(den).sort_values(by="oscrapovano").reset_index(drop=True)
            df_den.to_parquet(os.path.join(kam,nazev_souboru))

2024-10-31: 0
2024-11-01: 0
2024-11-02: 0
2024-11-03: 0
2024-11-04: 0
2024-11-05: 0
2024-11-06: 0
2024-11-09: 758
ar_Liberec_Pardubice_D0_2024-11-09_06-29-13.html
ar_Liberec_Pardubice_D0_2024-11-09_06-29-54.html
ar_Liberec_Pardubice_D0_2024-11-09_08-29-26.html
ar_Liberec_Pardubice_D0_2024-11-09_12-33-13.html
ar_Liberec_Pardubice_D0_2024-11-09_14-31-09.html
ar_Liberec_Pardubice_D0_2024-11-09_16-24-24.html
ar_Liberec_Pardubice_D0_2024-11-09_19-26-52.html
ar_Liberec_Pardubice_D0_2024-11-09_22-28-54.html
ar_Liberec_Pardubice_D0_2024-11-09_22-29-35.html
ar_Liberec_Pardubice_D10_2024-11-09_06-29-24.html
ar_Liberec_Pardubice_D10_2024-11-09_06-30-06.html
ar_Liberec_Pardubice_D10_2024-11-09_08-29-38.html
ar_Liberec_Pardubice_D10_2024-11-09_12-33-24.html
ar_Liberec_Pardubice_D10_2024-11-09_14-31-21.html
ar_Liberec_Pardubice_D10_2024-11-09_19-27-03.html
ar_Liberec_Pardubice_D10_2024-11-09_22-29-06.html
ar_Liberec_Pardubice_D10_2024-11-09_22-29-47.html
ar_Liberec_Pardubice_D11_2024-11-09_06-29-26.